In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from animalShelter import AnimalShelter


###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "SNHU1234"
CRUD = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(CRUD.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
#print(len(df.to_dict(orient='records')))
#print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

image_filename = 'Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H3("Derek Bamford-SNHU CS340- MongoDB Authentication"))),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'height':'20%', 'width':'20%'})),
    
    html.Hr(),
    html.Div(
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
# Reference https://dash.plotly.com/dash-core-components/dropdown
        dcc.Dropdown(
        id='filter-type',
          options=[
            {'label':'Reset', 'value': 'All'},
            {'label':'Water Rescue', 'value': 'Water'},
            {'label':'Mountain or Wilderness Rescue', 'value': 'Mountain'},
            {'label':'Disaster Rescue or Individual Tracking', 'value':'Disaster'},
        ],
            value='All', #Sets the default dropdown to 'All'
            multi=False, #Does not allow for multiple selections
            clearable=False, #We do not want the dropdown to have no value
        )
    ),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        
        editable=True,#makes the tables editable
        filter_action="native", #allows for filtering
        sort_action="native", #allows for sorting
        sort_mode="multi", #allows to sort more than one field
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0], # Set to zero To prevent issues with not selecting an initial row
        page_action="native",
        page_current=0, #sets start page to the beginning
        page_size=10 #sets rows per page

    ),
     html.Br(),
     html.Hr(),
    
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
     html.Div(
            id='map-id',
            className='col s12 m6',
            )
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################
@app.callback([Output('datatable-id','data'),
              Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
              
def update_dashboard(value):
## FIX ME Add code to filter interactive data table with MongoDB queries
    
    #Filter for water rescue, 
        if value == 'Water':
            print('Water Selected')
            df=pd.DataFrame.from_records(CRUD.read({
                "animal_type":"Dog",
                "breed": {"$in": ["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]},
                "sex_upon_outcome": "Intact Female",
                "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
            }))
            
    #Filter for Mountain rescue
        elif value == "Mountain":
            print('Mountian Selected')
            df=pd.DataFrame.from_records(CRUD.read({
                "animal_type":"Dog", 
                "breed":{"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog","Siberian Husky,Rottweiler"]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}   
            }))
        
    #Filter for Disaster Rescue
        elif value == "Disaster":
            print('Disaster Selected')
            df=pd.DataFrame.from_records(CRUD.read({
                "animal_type":"Dog", 
                "breed": {"$in": ["Doberman Pinscher", "German Shepherd","Golden Retriever","Bloodhound", "Rottweiler"]},
                "sex_upon_outcome": "Intact Male", 
                "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
            })) 
    
    #Reset the filters
        elif value == "Reset":
            print('Reset Selected')
            df.pd.DataFrame.from_records(CRUD.read({}))
            
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        return (data,columns)
        
# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])

def update_graphs(viewData):
    df = pd.DataFrame.from_records(viewData)
    print('Graph Activated')
    return [
        dcc.Graph(            
            figure = px.pie(df, names='breed', title='Preferred Animals')
        )    
    ]

#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])

def update_map(viewData, index):
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can 
    # be converted to a row index here
    if index is None:
        row = 0
    else:
        row = index[0]

# Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for 
            # the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                 ])
              ])
           ])
        ]

app.run_server(debug=True)

Connection Successful
Dash app running on http://127.0.0.1:10398/
Water Selected
Graph Activated
